In [8]:
import sqlite3

import pandas as pd

# Connect to SQLite database
sqlite_conn = sqlite3.connect('default.sqlite')
cursor = sqlite_conn.cursor()

# Get the list of all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()


In [9]:
tables

[('trulens_alembic_version',),
 ('trulens_feedback_defs',),
 ('trulens_records',),
 ('trulens_dataset',),
 ('trulens_ground_truth',),
 ('trulens_feedbacks',),
 ('trulens_apps',)]

In [10]:
import os

import snowflake.connector

snowflake_conn = snowflake.connector.connect(
    account =os.environ.get("SNOWFLAKE_ACCOUNT"),
    user=os.environ.get("SNOWFLAKE_USER"),
    password=os.environ.get("SNOWFLAKE_USER_PASSWORD"),
    database=os.environ.get("SNOWFLAKE_DATABASE"),
    schema='CONTEXT_RELEVANCE_TREC_2020',
    warehouse=os.environ.get("SNOWFLAKE_WAREHOUSE"),
    role=os.environ.get("SNOWFLAKE_ROLE"),
)
    

In [11]:
for table_name in tables:
    # Load data from SQLite into a DataFrame
    table_name = table_name[0]  # Extract table name from tuple
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", sqlite_conn)
    
    # Generate CREATE TABLE statement
    create_table_sql = f"CREATE OR REPLACE TABLE {table_name} ("
    columns = []
    for col_name, col_type in df.dtypes.items():
        if col_type == 'int64':
            col_type_snowflake = 'INTEGER'
        elif col_type == 'float64':
            col_type_snowflake = 'FLOAT'
        elif col_type == 'bool':
            col_type_snowflake = 'BOOLEAN'
        else:
            col_type_snowflake = 'TEXT'
        columns.append(f"{col_name} {col_type_snowflake}")
    create_table_sql += ", ".join(columns) + ");"
    
    # Create table in Snowflake
    cursor = snowflake_conn.cursor()
    cursor.execute(create_table_sql)


In [12]:
import os
import re

for table_name in tables:
    table_name = table_name[0]
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", sqlite_conn)
    
    # Save DataFrame as a CSV
    csv_file = f"{table_name}.csv"
    df.to_csv(csv_file, index=False)

    # Generate a Snowflake-compatible stage name by replacing special characters with underscores
    sanitized_stage_name = re.sub(r'\W+', '_', f"temp_stage_{table_name}")
    
    # Create a temporary stage in Snowflake
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute(f"CREATE OR REPLACE TEMPORARY STAGE {sanitized_stage_name}")
    
    # PUT the CSV file to the stage
    with open(csv_file, 'rb') as file_data:
        snowflake_cursor.execute(f"PUT file://{csv_file} @{sanitized_stage_name}")
    
    # Copy data from the stage to the table in Snowflake
    snowflake_cursor.execute(f"""
        COPY INTO {table_name}
        FROM @{sanitized_stage_name}
        FILE_FORMAT = (TYPE = 'CSV' FIELD_OPTIONALLY_ENCLOSED_BY = '"' SKIP_HEADER = 1)
    """)
    
    # Clean up temporary stage and local CSV file
    snowflake_cursor.execute(f"DROP STAGE IF EXISTS {sanitized_stage_name}")
    os.remove(csv_file)


In [13]:
sqlite_conn.close()
snowflake_conn.close()